In [1]:
!nvcc --version

!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Thu May 15 01:02:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8       

In [2]:
!curl -ssL https://magic.modular.com/ | bash

Installing the latest version of Magic...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 49.9M  100 49.9M    0     0  15.7M      0  0:00:03  0:00:03 --:--:-- 77.5M
Done. The 'magic' binary is in '/root/.modular/bin'

Two more steps:
1. To use 'magic', run this command so it's in your PATH:
source /root/.bashrc
2. To build with MAX and Mojo, go to http://modul.ar/get-started


In [3]:
import os
os.environ['PATH'] +=':/root/.modular/bin'

In [4]:
!magic init gpu_puzzles --format mojoproject

✔ Created /content/gpu_puzzles/mojoproject.toml


In [5]:
%cd gpu_puzzles/

/content/gpu_puzzles


In [58]:
%%writefile layout_basics.mojo
from gpu.host import DeviceContext
from layout import Layout, LayoutTensor

alias HEIGHT = 2
alias WIDTH = 3
alias dtype = DType.float32
alias layout = Layout.row_major(HEIGHT, WIDTH)
alias BLOCKS_PER_GRID = 1
alias THREADS_PER_BLOCK = 1


fn kernel[
    dtype: DType, layout: Layout
](tensor: LayoutTensor[mut=True, dtype, layout]):
    print("Before\n")
    print(tensor)
    tensor[0, 0] += 1.0
    print()
    print("After\n")
    print(tensor)


def main():
    ctx = DeviceContext(api="cuda")
    cpu_ctx = DeviceContext(api="cpu")
    buffer = ctx.enqueue_create_buffer[dtype](HEIGHT * WIDTH).enqueue_fill(0)
    cpu_buffer = cpu_ctx.enqueue_create_host_buffer[dtype](HEIGHT * WIDTH)

    for i in range(HEIGHT * WIDTH):
        cpu_buffer[i] = i**2

    cpu_buffer.enqueue_copy_to(buffer)

    tensor = LayoutTensor[mut=True, dtype, layout](buffer.unsafe_ptr())

    ctx.enqueue_function[kernel[dtype, layout]](
        tensor, grid_dim=BLOCKS_PER_GRID, block_dim=THREADS_PER_BLOCK
    )

    ctx.synchronize()

    print(ctx.name())
    print(ctx.api())
    print(cpu_ctx.api())
    cpu_buffer.unsafe_ptr()[] = 98.0
    print(cpu_buffer)


Overwriting layout_basics.mojo


In [59]:
!magic run mojo layout_basics.mojo

Before

0.0 1.0 4.0 
9.0 16.0 25.0 

After

1.0 1.0 4.0 
9.0 16.0 25.0 
Tesla T4
cuda
cpu
HostBuffer([98.0, 1.0, 4.0, 9.0, 16.0, 25.0])


In [57]:
!magic run mojo format layout_basics.mojo

reformatted layout_basics.mojo

All done! ✨ 🍰 ✨
1 file reformatted.
